In [1]:
import base64
import configparser
import multiprocessing as mp
import numpy as np
import os
import pandas as pd
import time
import json

from verta import Client

In [2]:
def load_imgs(n_imgs):
    path = 'images/'
    files = [file for file in sorted(os.listdir(path))][:n_imgs]
    data = []

    for file in files:
        with open(f"{path}{file}", 'rb') as img:
            img_bytes = base64.b64encode(img.read())
            img_str = img_bytes.decode('utf-8')
            img_str = json.dumps(img_str)
            img_str = np.array(img_str).tolist()
        
        data.append([file, img_str])

    return data

In [3]:
def show_metrics(n_imgs, n_threads, start_time, end_time):
    total_time = end_time - start_time
    total_time = time.strftime('%Mm %Ss', time.gmtime(total_time))
    
    print(f'Images processed: {n_imgs}.')
    print(f'Threads: {n_threads}.')
    print(f'Total time: {total_time}.')

In [4]:
def save_results(results, n_imgs):
    cols = list(results[0].keys())[:-1]
    cols.extend(list(results[0]['bboxes'].keys()))
    data = []

    for item in results:
        values = list(item.values())[0:3]
        values.extend(list(item['bboxes'].values()))
        data.append(values)

    df = pd.DataFrame(data, columns=cols)
    df.to_csv(f"results/{n_imgs}.csv", index=False)

In [5]:
def main():
    config = configparser.ConfigParser()
    config.read('config.ini')

    VERTA_HOST = config['APP']['VERTA_HOST']
    ENDPOINT_NAME = config['APP']['ENDPOINT_NAME']

    os.environ['VERTA_EMAIL'] = config['APP']['VERTA_EMAIL']
    os.environ['VERTA_DEV_KEY'] = config['APP']['VERTA_DEV_KEY']

    client = Client(VERTA_HOST, debug=True)
    endpoint = client.get_or_create_endpoint(ENDPOINT_NAME)
    model = endpoint.get_deployed_model()

    n_threads = int(mp.cpu_count())
    n_imgs = 5  
    imgs = load_imgs(n_imgs)
        
    start_time = time.time()
    pool = mp.Pool(n_threads)
    map_results = pool.map_async(model.predict, imgs, chunksize=1)

    while not map_results.ready():
        print(f"Images remaining: {map_results._number_left}")
        time.sleep(5)

    results = map_results.get()
    pool.close()
    pool.join()
    end_time = time.time()

    show_metrics(n_imgs, n_threads, start_time, end_time)
    save_results(results, n_imgs)

In [6]:
if __name__ == '__main__':
    main()

got VERTA_EMAIL from environment
got VERTA_DEV_KEY from environment
[DEBUG] using credentials: EmailCredentials(caio@verta.ai, 68b041eb-****-****-****-************)
connection successfully established
got existing Endpoint: object-detection-base64
Images remaining: 5


2022-11-29 18:00:14.965533: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 18:00:14.966730: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 18:00:14.968050: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

Images processed: 5.
Threads: 16.
Total time: 00m 05s.
